In [5]:
from __future__ import print_function
from __future__ import division

#from sklearn.datasets import make_classification
#from sklearn.cross_validation import cross_val_score
#from sklearn.ensemble import RandomForestClassifier as RFC
#from sklearn.svm import SVC
import string
import re
import random

import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchviz import make_dot, make_dot_from_trace

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle


from bayes_opt import BayesianOptimization

In [6]:
import model_gp as model
import Loaddata as Loaddata
import TrainVaTe as TVT

# check GPU availability
use_cuda = torch.cuda.is_available()
use_cuda

True

In [7]:
# Load data set and target values
"""
data, target = make_classification(
    n_samples=1000,
    n_features=45,
    n_informative=12,
    n_redundant=7
)
"""
set_x = pickle.load(open('Data/h143.visits', 'rb'), encoding='bytes')
set_y = pickle.load(open('Data/h143.labels', 'rb'),encoding='bytes')

model_x = []
for patient in set_x:
    model_x.append([each for visit in patient for each in visit])  
    
merged_set= [[set_y[i],model_x[i]] for i in range(len(set_y))] #list of list or list of lists of lists
print("\nLoading and preparing data...")    
train1, valid1, test1 = Loaddata.load_data(merged_set)
print("\nSample data after split:")  
print(train1[0])
print("model is", 'LR') #can change afterwards


Loading and preparing data...

Sample data after split:
[0, [1667, 144, 62, 85, 1667, 144, 62, 85]]
model is LR


In [19]:
#Hyperparamters to tune for LR: embed_dim, l2, lr. Define a function to return the validation AUC of the model 
def LR_tune(l2, lr):
    #little transformations to use the searched values
    l2 = np.exp(l2)
    lr = np.exp(lr)
    ehr_model = model.EHR_LR()  
    if use_cuda:
        ehr_model = ehr_model.cuda()
    optimizer = optim.Adam(ehr_model.parameters(), lr=lr, weight_decay=l2)
    current_loss_allep=[]
    all_losses_allep=[]
    avg_losses_allep=[]
    #train_auc_allep =[]
    valid_auc_allep =[]
    test_auc_allep=[]
    
    for ep in range(20):
        #start = time.time()
        current_loss, train_loss = TVT.train(train1, model= ehr_model, optimizer = optimizer, batch_size = 1) #mb=args.mb
        avg_loss = np.mean(train_loss)
        #train_time = timeSince(start)
        #print ('\n Current running on: Epoch ', ep,'Training loss:',' Average loss', avg_loss)
        #print(train_loss, '\n train loss plot')
        #TVT.showPlot(train_loss)
        #eval_start = time.time()
        #train_auc, y_real, y_hat = TVT.calculate_auc(model= ehr_model, data = train1, which_model ='LR', batch_size = 1)
        #print ('\n Current running on: Epoch ', ep,' Training auc:', train_auc)
        #TVT.auc_plot(y_real, y_hat)
        valid_auc, y_real, y_hat  = TVT.calculate_auc(model = ehr_model, data = valid1, which_model = 'LR', batch_size = 1)
        #print ('\n Current running on: Epoch ', ep,' validation auc:', valid_auc)
        #TVT.auc_plot(y_real, y_hat)
        test_auc, y_real, y_hat = TVT.calculate_auc(model = ehr_model, data = test1, which_model = 'LR', batch_size = 1)
        #print ('\n Current running on: Epoch ', ep,' test auc:', test_auc)
        #TVT.auc_plot(y_real, y_hat)
        #eval_time = timeSince(eval_start)
        #print ("Epoch ", ep, "Summary:  Training_auc :", train_auc, " , Validation_auc : ", valid_auc, " ,& Test_auc : " , test_auc, " Avg Loss: ", avg_loss )
        current_loss_allep.append(current_loss)
        all_losses_allep.append(train_loss)
        avg_losses_allep.append(avg_loss)
        #train_auc_allep.append(train_auc)
        valid_auc_allep.append(valid_auc)
        test_auc_allep.append(test_auc)
        final_max_valid = max(valid_auc_allep)
    return final_max_valid

In [20]:
if __name__ == "__main__":
    gp_params = {"alpha": 1e-4}

    LRBO = BayesianOptimization(LR_tune,
        {'l2': (-16, 1), 'lr': (-11, -2) })
    LRBO.explore({'l2': [-11], 'lr': [-9]})

    LRBO.maximize(n_iter=20, **gp_params)

    print('-' * 53)
    print('Final Results')
    print('LR: %f' % LRBO.res['max']['max_val'])

Initialization
-----------------------------------------------------
 Step |   Time |      Value |        l2 |        lr | 
    1 | 21m00s |    0.76808 |  -11.0000 |   -9.0000 | 
    2 | 18m09s |    0.72876 |   -9.2810 |   -3.8796 | 
    3 | 16m18s |    0.73583 |   -3.2857 |   -9.4977 | 
    4 | 17m49s |    0.65022 |   -5.4683 |   -2.9056 | 
    5 | 20m02s |    0.52860 |   -0.4957 |   -8.2555 | 
    6 | 18m06s |    0.61068 |   -1.5312 |   -6.7958 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |        l2 |        lr | 
    7 | 15m35s |    0.60945 |  -16.0000 |   -2.0000 | 
    8 | 15m37s |    0.63542 |  -16.0000 |  -11.0000 | 
    9 | 15m10s |    0.61660 |   -7.5678 |  -11.0000 | 
   10 | 15m35s |    0.75438 |  -16.0000 |   -6.5960 | 
   11 | 15m47s |    0.77169 |   -6.6973 |   -6.9413 | 
   12 | 15m33s |    0.76444 |  -12.7978 |   -6.1913 | 
   13 | 15m23s |    0.78146 |   -9.8993 |   -6.9196 | 
   14 | 15m53s |    0.76388 | 